BU PROJEDE 2020-2021 SEZONU INGILTERE PREMIER LEAGUE'de OYNAYAN FORVET OYUNCULARININ GOLLERİNİ TAHMİN EDECEĞİZ

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("ShootingStatsFw.csv")
pd.set_option('display.max_rows', df.shape[0]+1)
df = df[(df['SoT']>2) & (df['Avg90s']>2)]
df

,Rk,Player,Nation,Pos,Squad,Age,Born,Avg90s,Gls,Sh,...,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,2,Tammy Abraham,ENG,FW,Chelsea,22,1997,11.6,6,32,...,0.46,9.7,0,0,0,6.4,6.4,0.20,-0.4,-0.4
1,3,Che Adams,SCO,FW,Southampton,24,1996,29.6,9,56,...,0.29,13.1,0,0,0,8.8,8.8,0.16,0.2,0.2
2,6,Sergio Agüero,ARG,FWMF,Manchester City,32,1988,6.2,4,17,...,0.30,13.9,0,1,2,3.4,1.8,0.11,0.6,1.2
4,28,Michail Antonio,JAM,FW,West Ham,30,1990,21.9,10,64,...,0.43,12.8,0,0,0,11.4,11.4,0.18,-1.4,-1.4
5,32,Pierre-Emerick Aubameyang,GAB,FW,Arsenal,31,1989,25.9,10,56,...,0.42,13.5,0,2,2,10.6,9.1,0.16,-0.6,-1.1
7,35,Jordan Ayew,GHA,FWMF,Crystal Palace,28,1991,23.3,1,25,...,0.09,16.7,0,0,0,2.4,2.4,0.09,-1.4,-1.4
8,41,Gareth Bale,WAL,FW,Tottenham,31,1989,10.2,11,38,...,0.58,18.2,7,0,0,5.3,5.3,0.15,5.7,5.7
9,42,Patrick Bamford,ENG,FW,Leeds United,26,1993,33.9,17,105,...,0.33,12.7,0,2,2,17.3,15.7,0.15,-0.3,-0.7
10,46,Ashley Barnes,ENG,FW,Burnley,30,1989,14.8,3,39,...,0.17,15.7,0,1,1,5.1,4.4,0.11,-2.1,-2.4
11,47,Harvey Barnes,ENG,FWMF,Leicester City,22,1997,21.6,9,55,...,0.36,16.9,0,0,0,6.2,6.2,0.11,2.8,2.8


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 133
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Rk       94 non-null     int64  
 1   Player   94 non-null     object 
 2   Nation   94 non-null     object 
 3   Pos      94 non-null     object 
 4   Squad    94 non-null     object 
 5   Age      94 non-null     int64  
 6   Born     94 non-null     int64  
 7   Avg90s   94 non-null     float64
 8   Gls      94 non-null     int64  
 9   Sh       94 non-null     int64  
 10  SoT      94 non-null     int64  
 11  SoT%     94 non-null     float64
 12  Sh/90    94 non-null     float64
 13  SoT/90   94 non-null     float64
 14  G/Sh     94 non-null     float64
 15  G/SoT    94 non-null     float64
 16  Dist     94 non-null     float64
 17  FK       94 non-null     int64  
 18  PK       94 non-null     int64  
 19  PKatt    94 non-null     int64  
 20  xG       94 non-null     float64
 21  npxG     94 non-n

ilk olarak csv dosyamızdan verilerimizi çekip kaç satır ve sütun olduğunu görmek ve gol tahmini için almak istediğim (kendimce mantıklı bulduğum) değerleri alıyorum ve filtrelenmiş bir şekilde
yeni bir dataframe'e aktarıyorum

Bu projede KNN algoritmasını kullanarak tahminimizi gerçekleştireceğiz

In [18]:
shotOnly= df[ (df['Gls']!=0)][['Player','Nation','Pos','Squad','Avg90s',
          'Sh','SoT','Dist','xG',
        'Gls']].sort_values('Gls',ascending=False)

pd.set_option('display.max_rows', shotOnly.shape[0]+1)
shotOnly

,Player,Nation,Pos,Squad,Avg90s,Sh,SoT,Dist,xG,Gls
62,Harry Kane,ENG,FW,Tottenham,34.2,134,47,17.7,20.5,23
104,Mohamed Salah,EGY,FW,Liverpool,34.2,120,46,15.8,20.9,22
9,Patrick Bamford,ENG,FW,Leeds United,33.9,105,45,12.7,17.3,17
47,Son Heung-min,KOR,FW,Tottenham,34.6,68,35,17.0,10.3,17
19,Dominic Calvert-Lewin,ENG,FW,Everton,31.9,81,46,11.7,15.4,16
120,Jamie Vardy,ENG,FW,Leicester City,31.6,73,27,14.1,19.7,15
124,Ollie Watkins,ENG,FW,Aston Villa,37.0,98,45,13.0,16.0,14
66,Alexandre Lacazette,FRA,FW,Arsenal,21.4,43,25,14.2,9.7,13
50,Kelechi Iheanacho,NGA,FW,Leicester City,16.2,58,25,17.1,7.8,12
129,Chris Wood,NZL,FW,Burnley,30.5,67,37,13.1,13.3,12


Dataframe mimizdeki futbolcu isimleri, ülkeleri, oynadıkları pozisyon ve takım gibi sözel bilgilerin olmadığı tüm verilerimizi ayrı bir dataframe e aktarıyorum

In [19]:
dataGoals=shotOnly[['Avg90s','Gls','SoT','Sh','xG','Dist']]
dataGoals.sort_values('Avg90s',ascending=False)

,Avg90s,Gls,SoT,Sh,xG,Dist
124,37.0,14,45,98,16.0,13.0
47,34.6,17,35,68,10.3,17.0
62,34.2,23,47,134,20.5,17.7
104,34.2,22,46,120,20.9,15.8
9,33.9,17,45,105,17.3,12.7
95,32.4,11,36,79,9.3,18.4
19,31.9,16,46,81,15.4,11.7
97,31.8,7,32,82,11.5,14.5
120,31.6,15,27,73,19.7,14.1
32,31.5,9,32,83,12.6,13.8


Yukarıda da belirttiğim gibi çıkış noktamız (bağımlı veriler) oyuncuların attığı goller ['Gls'] y değerine, onun dışındaki tüm bağımsız verileri x değerine atıyoruz

In [22]:
X=dataGoals.drop(['Gls'],axis=1).values
y=dataGoals['Gls'].values

In [23]:
X_train, X_test, y_train, y_test = train_test_split (X,y, test_size=0.20, random_state=35)

Verilerimizi Train ve Test olarak ayırdığımıza göre modellemeye başlayabiliriz. Vira Bismillah

In [27]:
knn_model = KNeighborsRegressor()

In [28]:
knn_model

KNeighborsRegressor()

In [29]:
model = knn_model.fit(X_train, y_train)

In [30]:
model

KNeighborsRegressor()

In [31]:
shotX = shotOnly[['Avg90s', 'Sh','SoT','Dist','xG']].values
prGls = model.predict(shotX)
prGls1 = np.around(prGls, 1)
prGls1

array([13.6, 13.6, 12.8, 11.8, 12.8, 10.6, 12.8,  6.8,  7.8, 11.8,  4.6,
        6.4, 12.8, 13.2,  5. ,  6.4,  7.8,  7.2,  8. ,  7.8,  9.2,  7.8,
        3. ,  8.2,  7.8,  7.2,  5.6,  7.8,  5.6, 12.6,  8. ,  4.8,  8.6,
        7.8,  3. , 12.6,  6.4, 12.6,  3. ,  4. ,  5.6,  3.2,  3. ,  6.4,
        6.4,  6.4,  3.6,  4. ,  6. ,  7. ,  2.2,  5.6,  2. ,  2.2,  5.6,
        2.2,  2.2,  3. ,  2.2,  1.8,  3.6,  3.2,  3. ,  2.2,  7.2,  1.8,
        2.2,  3. ,  2. ,  3.4,  2. ,  3. ,  2. ,  2.2,  2.2,  2.2,  2.2,
        3.2,  2.4,  2.4,  2.2,  3. ,  2.2,  2.2,  2.2,  2.2,  1.8,  2.2,
        2.2,  2.2,  2. ])

Yukarıda ilk başta knn modelimizi oluşturup, daha sonra modelimizi eğittik. 

Sonrasında dataframe deki bağımsız değişkenleri geçici olarak bir dataframe e ekledik ve bu dataframe üzerinden oluşturduğumuz modele göre tahminlerimizi gerçekleştirdik

Aşağıda da oluştuduğumuz dataframe e tahmin sonuçlarımızı ve tahmin ile bağımlı değerlerin birbirine olan farklarının sonucunu da eklemiş olduk

In [32]:
shotOnly['PredGls'] = prGls1
shGls = shotOnly['Gls']
prGls2 = shotOnly['PredGls']
shotOnly['Diff'] = prGls2 - shGls
shotOnly.sort_values('PredGls',ascending=False)

,Player,Nation,Pos,Squad,Avg90s,Sh,SoT,Dist,xG,Gls,PredGls,Diff
62,Harry Kane,ENG,FW,Tottenham,34.2,134,47,17.7,20.5,23,13.6,-9.4
104,Mohamed Salah,EGY,FW,Liverpool,34.2,120,46,15.8,20.9,22,13.6,-8.4
95,Marcus Rashford,ENG,FW,Manchester Utd,32.4,79,36,18.4,9.3,11,13.2,2.2
9,Patrick Bamford,ENG,FW,Leeds United,33.9,105,45,12.7,17.3,17,12.8,-4.2
19,Dominic Calvert-Lewin,ENG,FW,Everton,31.9,81,46,11.7,15.4,16,12.8,-3.2
124,Ollie Watkins,ENG,FW,Aston Villa,37.0,98,45,13.0,16.0,14,12.8,-1.2
73,Sadio Mané,SEN,FW,Liverpool,31.2,93,35,12.8,15.2,11,12.8,1.8
97,Richarlison,BRA,FW,Everton,31.8,82,32,14.5,11.5,7,12.6,5.6
32,Roberto Firmino,BRA,FW,Liverpool,31.5,83,32,13.8,12.6,9,12.6,3.6
126,Timo Werner,GER,FW,Chelsea,28.9,80,31,13.8,11.9,6,12.6,6.6


In [35]:
y_pred = model.predict(X_test)

In [37]:
np.sqrt(mean_squared_error(y_test, y_pred))

3.152275702609649

Yukarıda test hatasını öğrendik. Sırada eğitim hatasını öğrenmek var (RMSE ÜZERINDEN HESAPLANMIŞTIR)

In [38]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

2.2593017603774057